In [2]:
cd ..

/Users/emekgozluklu/Desktop/IDP/Repositories/tutor


/Users/emekgozluklu/Desktop/IDP/Repositories/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [16]:
from IPython.display import display, Math, Latex, clear_output
from src.model.GPT import GPTModel, COSTS
from src.datasets.MATH import MATH, MATHRecord
import datetime
import json
import random
import openai as oai
from openai import OpenAI
from copy import copy
from tqdm import tqdm
import itertools
import pickle

random.seed(52)

# Prepare dataset
dataset = MATH('data/math.jsonl')
dataset.records = [rec for rec in dataset.records if len(rec.formatted_solution) < 750]  # to cut the token
random.shuffle(dataset.records)

# Prepare model
gpt3 = GPTModel(model="gpt-3.5-turbo-0125", temperature=0.7, max_tokens=500)
gpt4 = GPTModel(model="gpt-4o", temperature=1, max_tokens=500)

gpt4.model = "gpt-4o"
COSTS['gpt-4o'] = {'input': 5, 'output': 15}

file_train = 'output/final_checked_fine_tuning.pkl'
file_eval = 'output/evaluation_dataset/eval_v2_full.pkl'

data_train = pickle.load(open(file_train, 'rb'))
data_eval = pickle.load(open(file_eval, 'rb'))

In [22]:
def show_f(out):
    display(Latex(r"$\textbf{Question " +str(out['record'].id) + ": }$"))
    display(Latex(r"$\textbf{Question: }$"))
    display(Latex(out['record'].question))
    display(Latex(r"$\textbf{Solution: }$"))
    display(Latex(out['record'].formatted_solution))
    display(Latex(r"$\textbf{Error type: }$" + out['error_type']))
    display(Latex((r"$\textbf{ERROR EXPLANATION: }$"  + out['error_explanation'])))
    display(Latex((r"$\textbf{STUDENT'S SOLUTION: }$"  + out['incorrect_solution'])))


    display(Latex("================================================"))

    display(Latex("================================================"))
    display(Latex((r"$\textbf{Feedback 1: }$"  + out['feedbacks']['zeroshot_default'])))
    display(Latex((r"$\textbf{Feedback 2: }$"  + out['feedbacks']['zeroshot_wo_explanation'])))
    display(Latex((r"$\textbf{Feedback 3: }$"  + out['feedbacks']['zeroshot_with_explanation'])))


In [23]:
feedback_prompt_zeroshot_default = '''Student made a mistake while attempting to solve this question. Considering the correct answer and student's solution, generate a feedback for the student.

QUESTION: question_input
CORRECT SOLUTION: correct_input
STUDENT'S SOLUTION: student_input
FEEDBACK: 
'''

# ----
feedback_prompt_concentration_zeroshot_wo_explanation = '''Student made a concentration error in this question. A concentration error causes from lapse in attention leads to an oversight or a  careless mistake in the calculations or steps. Considering the correct answer and student's solution, generate a feedback for the student.

QUESTION: question_input
CORRECT SOLUTION: correct_input
STUDENT'S SOLUTION: student_input
ERROR TYPE: CONCENTRATION
FEEDBACK: 
'''

feedback_prompt_logical_zeroshot_wo_explanation = '''Student made a logical error in this question. A logical error is caused from a mistake made in reasoning or in the application of a rule, even though the basic operations are performed correctly. Considering the correct answer and student's solution, generate a feedback for the student.

QUESTION: question_input
CORRECT SOLUTION: correct_input
STUDENT'S SOLUTION: student_input
ERROR TYPE: LOGICAL
FEEDBACK: 
'''
# ----
feedback_prompt_concentration_zeroshot_with_explanation = '''Student made a concentration error in this question. A concentration error causes from lapse in attention leads to an oversight or a  careless mistake in the calculations or steps. Considering the correct answer and student's solution, generate a feedback for the student.

QUESTION: question_input
CORRECT SOLUTION: correct_input
STUDENT'S SOLUTION: student_input
ERROR TYPE: CONCENTRATION
ERROR EXPLANATION: explanation_input
FEEDBACK: 
'''

feedback_prompt_logical_zeroshot_with_explanation = '''Student made a logical error in this question. A logical error is caused from a mistake made in reasoning or in the application of a rule, even though the basic operations are performed correctly. Considering the correct answer and student's solution, generate a feedback for the student.

QUESTION: question_input
CORRECT SOLUTION: correct_input
STUDENT'S SOLUTION: student_input
ERROR TYPE: LOGICAL
ERROR EXPLANATION: explanation_input
FEEDBACK: 
'''


In [24]:
# GENERATE FEEDBACKS

feedbacks = {}
feedbacks['zeroshot_default'] = []
feedbacks['zeroshot_wo_explanation'] = []
feedbacks['zeroshot_with_explanation'] = []

In [94]:
# run zeroshot_default

for rec in tqdm(data_train[10:30]):

    # update rec for feedbacks
    rec['feedbacks'] = {}
    
    p = feedback_prompt_zeroshot_default
    p = p.replace('question_input', rec['record'].question)
    p = p.replace('correct_input', rec['record'].formatted_solution)
    p = p.replace('student_input', rec['incorrect_solution'])
    
    out = gpt3.query(p)[0]
    feedbacks['zeroshot_default'].append(out)
    rec['feedbacks']['zeroshot_default'] = out


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:39<00:00,  1.97s/it]


In [95]:
# run zeroshot_wo_explanation

for rec in tqdm(data_train[10:30]):

    if rec['error_type'] == 'CONCENTRATION':
        p = feedback_prompt_concentration_zeroshot_wo_explanation
    elif rec['error_type'] == 'LOGICAL':
        p = feedback_prompt_logical_zeroshot_wo_explanation
    else:
        p = feedback_prompt_zeroshot_default
        
    p = p.replace('question_input', rec['record'].question)
    p = p.replace('correct_input', rec['record'].formatted_solution)
    p = p.replace('student_input', rec['incorrect_solution'])

    
    out = gpt3.query(p)[0]
    feedbacks['zeroshot_wo_explanation'].append(out)
    rec['feedbacks']['zeroshot_wo_explanation'] = out

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:38<00:00,  1.93s/it]


In [96]:
# run zeroshot_with_explanation

for rec in tqdm(data_train[10:30]):

    if rec['error_type'] == 'CONCENTRATION':
        p = feedback_prompt_concentration_zeroshot_with_explanation
    elif rec['error_type'] == 'LOGICAL':
        p = feedback_prompt_logical_zeroshot_with_explanation
    else:
        p = feedback_prompt_zeroshot_default
        
    p = p.replace('question_input', rec['record'].question)
    p = p.replace('correct_input', rec['record'].formatted_solution)
    p = p.replace('student_input', rec['incorrect_solution'])
    p = p.replace('explanation_input', rec['error_explanation'])
    
    out = gpt3.query(p)[0]
    feedbacks['zeroshot_with_explanation'].append(out)
    rec['feedbacks']['zeroshot_with_explanation'] = out

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:36<00:00,  1.81s/it]


In [28]:
selection = []
for rec in (data_train[:10]):
    show_f(rec)
    selection.append(input('best? : '))
    


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>


KeyboardInterrupt



In [99]:
show_f(data_train[13])

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [63]:
data_train[13]['incorrect_solution'] = "$31_8=3\\cdot8^2+1\\cdot8^1=192+8=\\boxed{200}$."



In [93]:
show_f(data_train[])

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [66]:
out = data_train[0]
display(Latex(r"$\textbf{Question: }$"))
display(Latex(out['record'].question))
display(Latex(r"$\textbf{Solution: }$"))
display(Latex(out['record'].formatted_solution))
display(Latex((r"$\textbf{STUDENT'S SOLUTION: }$")))
display(Latex(out['incorrect_solution']))
display(Latex(r"$\textbf{Error type: }$" + out['error_type']))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [91]:
out = data_train[1]
display(Latex(r"$\textbf{Question: }$"))
display(Latex(out['record'].question))
display(Latex(r"$\textbf{Solution: }$"))
display(Latex(out['record'].formatted_solution))
display(Latex(r"$\textbf{Error type: }$" + out['error_type']))
display(Latex(r"$\textbf{Potential error: }$" + out['error_explanation']))

display(Latex((r"$\textbf{Student's Solution: }$")))



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [68]:
out

{'record': MATH | Record: 1780 	 Question: Determine the number of ways to arrange the letters of the word MADAM.,
 'error_type': 'CONCENTRATION',
 'error_explanation': 'The student may have counted the total letters incorrectly. The student should have counted 5 instead of 4.',
 'incorrect_solution': "First we count the arrangements if all the letters are unique, which is $4!$. Then since the M's and the A's are not unique, we divide by $2!$ twice for the arrangements of M's and the arrangements of A's, for an answer of $\\dfrac{4!}{2! \\times 2!} = \\boxed{12}$.",
 'feedbacks': {'zeroshot_default': 'Good attempt, but it seems like you made a mistake in counting the initial number of arrangements. The word MADAM has 5 unique letters, not 4, so the correct number of arrangements if all the letters are unique is $5!$, not $4!$. Therefore, the correct answer should be $\\dfrac{5!}{2! \\times 2!} = 30$, not 12. Remember to carefully count the total number of arrangements in the beginning 

In [119]:
out = data_train[13]

display(Latex(r"$\textbf{Question: }$" + out['record'].question))
display(Latex(r"$\textbf{Solution: }$" + out['record'].formatted_solution))
display(Latex((r"$\textbf{STUDENT'S SOLUTION: }$"  + out['incorrect_solution'])))


display(Latex("================================================"))

#display(Latex(r"$\textbf{Without Error Detection: }$"))
#display(Latex(out['feedbacks']['zeroshot_default']))
#display(Latex(r"$\textbf{Only Error Type: }$"))
#display(Latex(out['feedbacks']['zeroshot_wo_explanation']))
display(Latex(r"$\textbf{Error Type + Explanation: }$"))
display(Latex(out['feedbacks']['zeroshot_with_explanation']))
display(Latex('...'))
display(Latex('...'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [122]:
display(Latex(r"$\textbf{Only Error Type: }$"))
display(Latex(out['feedbacks']['zeroshot_wo_explanation']))
display(Latex(r"$\textbf{Error Type + Explanation: }$"))
display(Latex(out['feedbacks']['zeroshot_with_explanation']))
display(Latex(''))
display(Latex(''))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [143]:
out = data_train[25]

out['record'].question


'Container A holds 4 red balls and 6 green balls; containers B and C each hold 6 red balls and 4 green balls. A container is selected at random and then a ball is randomly selected from that container. What is the probability that the ball selected is green? Express your answer as a common fraction.'

In [146]:
gpt3.query('Translate all the question text to Turkish. Make sure that the logic and the equations in the question do not change.\nQ:' + data_train[25]['record'].question)

['S: A Düşesi 4 kırmızı top ve 6 yeşil top içerir; B ve C Düşeleri ise herbirinde 6 kırmızı top ve 4 yeşil top bulunmaktadır. Rastgele bir düşe seçilir ve daha sonra bu düşeden rastgele bir top seçilir. Seçilen topun yeşil olma olasılığı nedir? Cevabınızı basit kesir olarak ifade edin.']